In [1]:
import os

In [13]:
import os
import sys

# Get the absolute path to the project root
project_root = "/Users/saiaakash/Documents/mlops/DS_main"

# Add the src directory to the Python path
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Print to verify
print(f"Project root: {project_root}")
print(f"Python path: {sys.path}")

Project root: /Users/saiaakash/Documents/mlops/DS_main
Python path: ['/Users/saiaakash/Documents/mlops/DS_main/src', '/Users/saiaakash/Documents/mlops/src', '/opt/anaconda3/envs/airflow-env/lib/python311.zip', '/opt/anaconda3/envs/airflow-env/lib/python3.11', '/opt/anaconda3/envs/airflow-env/lib/python3.11/lib-dynload', '', '/opt/anaconda3/envs/airflow-env/lib/python3.11/site-packages']


In [29]:
# Return to project root
os.chdir("/Users/saiaakash/Documents/mlops/DS_main")
%pwd  # Verify we're back in the correct directory

'/Users/saiaakash/Documents/mlops/DS_main'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
!pip install joblib python-box ensure

In [14]:
from data_science.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from data_science.utils.common import read_yaml, create_directories

In [30]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,  # Changed from config.source.url
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


In [31]:
import os
import urllib.request as request
from data_science import logger
import zipfile
from box import ConfigBox

In [32]:
class DataIngestion:
    def __init__(self,config:ConfigBox):
        self.config = config
    #donwloding the zip file
    def download_file(self):
        if not os.path.exists(self.config.data_source):
            file,headers=request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.data_source
            )
            logger.info(f"Downloaded file from {self.config.source_url} to {self.config.data_source}")
        else:
            logger.info(f"File already exists at {self.config.data_source}")
        
        #extracting the zip file
        def extract_zip_file(self):
            unzip_path=self.config.unzip_dir
            os.makedirs(unzip_path,exist_ok=True)
            with zipfile.ZipFile(self.config.data_source, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)


In [34]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"Error in data ingestion stage: {e}")
    


[2025-07-28 08:56:38,227: INFO: common]: yaml file: config/config.yaml loaded successfully
[2025-07-28 08:56:38,283: INFO: common]: yaml file: params.yaml loaded successfully
[2025-07-28 08:56:38,352: INFO: common]: yaml file: schema.yaml loaded successfully
[2025-07-28 08:56:38,358: INFO: common]: created directory at: artifacts
[2025-07-28 08:56:38,359: INFO: common]: created directory at: artifacts/data_ingestion
[2025-07-28 08:56:38,359: ERROR: 1211084895]: Error in data ingestion stage: 'DataIngestionConfig' object has no attribute 'data_source'
